In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer
import os
import matplotlib.colors as mcolors
import xarray as xr
import netCDF4 as nc
from netCDF4 import Dataset

In [40]:
#ACQUISITION

#All coral data files are from MAREANO
gml_file_coral_reefs_trondelag = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_50_Trondelag_25832_Korallrev_GML.gml"
gdf_coral_reefs_trondelag = gpd.read_file(gml_file_coral_reefs_trondelag)

gml_file_coral_reefs_more_og_romsdal = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_15_More_og_Romsdal_25832_Korallrev_GML.gml"
gdf_coral_reefs_more_og_roms = gpd.read_file(gml_file_coral_reefs_more_og_romsdal)

gml_file_coral_reefs_nordland = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_18_Nordland_25833_Korallrev_GML.gml"
gdf_coral_reefs_nordland = gpd.read_file(gml_file_coral_reefs_nordland)

gml_file_coral_reefs_vestland = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_46_Vestland_25832_Korallrev_GML.gml"
gdf_coral_reefs_vestland = gpd.read_file(gml_file_coral_reefs_vestland)

gml_file_coral_reefs_troms = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_55_Troms_25833_Korallrev_GML.gml"
gdf_coral_reefs_troms = gpd.read_file(gml_file_coral_reefs_troms)

gml_file_coral_reefs_finnmark = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_56_Finnmark_25833_Korallrev_GML.gml"
gdf_coral_reefs_finnmark = gpd.read_file(gml_file_coral_reefs_finnmark)

gml_file_coral_reefs_norskehavet = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_63_Norskehavet_25832_Korallrev_GML.gml"
gdf_coral_reefs_norskehavet = gpd.read_file(gml_file_coral_reefs_norskehavet)

gml_file_coral_reefs_barentshavet = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_64_Barentshavet_vest_25833_Korallrev_GML.gml"
gdf_coral_reefs_barentshavet = gpd.read_file(gml_file_coral_reefs_barentshavet)


#Define file paths to merge
gml_files = {
    "Trondelag": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_50_Trondelag_25832_Korallrev_GML.gml",
    "More_og_Romsdal": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_15_More_og_Romsdal_25832_Korallrev_GML.gml",
    "Nordland": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_18_Nordland_25833_Korallrev_GML.gml",
    "Vestland": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_46_Vestland_25832_Korallrev_GML.gml",
    "Troms": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_55_Troms_25833_Korallrev_GML.gml",
    "Finnmark": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_56_Finnmark_25833_Korallrev_GML.gml",
    "Norskehavet": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_63_Norskehavet_25832_Korallrev_GML.gml",
    "Barentshavet": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_64_Barentshavet_vest_25833_Korallrev_GML.gml"
}

#Read all files into a list of GeoDataFrames
gdfs = [gpd.read_file(file).to_crs("EPSG:4326") for file in gml_files.values()]

#Combine
gdf_all_coral_files = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))


In [41]:
#CLEANING
#remove any entries with nan values for longitude or latitude (none of the points used here had this)
gdf_all_coral_files = gdf_all_coral_files.dropna(subset=['lengdegrad', 'breddegrad'])

In [45]:
#Load the original SINMOD data from SINMOD grid

filename_SINMOD_original_grid = "/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc"

sinmod_data = Dataset(filename_SINMOD_original_grid, 'r')


In [43]:
#Remove coral points outside model area (change limits based on lon/lat range of SINMOD model area)

gdf_all_coral_reefs = gdf_all_coral_files[
    (gdf_all_coral_files['lengdegrad'] >= -1.9012374877929688) &
    (gdf_all_coral_files['lengdegrad'] <= 15.060638427734375) &
    (gdf_all_coral_files['breddegrad'] >= 59.99595260620117) &
    (gdf_all_coral_files['breddegrad'] <= 67.96094512939453)
]


In [47]:
#Transformation of coral coordinates to SINMOD grid coordinates

def obtain_sinmod_crs(sinmod_data):
    
    grid_mapping = sinmod_data.variables['grid_mapping']

    horizontal_resolution = grid_mapping.getncattr('horizontal_resolution')

    print(f"Horizontal resolution: {horizontal_resolution} meters")

    crs_sinmod = CRS.from_proj4(f"+proj=stere "
                                f"+lat_0={grid_mapping.latitude_of_projection_origin} "
                                f"+lat_ts={grid_mapping.standard_parallel} "
                                f"+lon_0={grid_mapping.straight_vertical_longitude_from_pole} "
                                f"+x_0={grid_mapping.false_easting} "
                                f"+y_0={grid_mapping.false_northing} "
                                f"+a={grid_mapping.semi_major_axis} "
                                f"+b={grid_mapping.semi_minor_axis} "
                                f"+units=m +no_defs")
    return crs_sinmod


crs_sinmod = obtain_sinmod_crs(sinmod_data)

Horizontal resolution: 800.0 meters


In [48]:
#Next, transform all coral points to SINMOD grid coordinates

def transform_coral_coordinates(coral_data, crs_sinmod):
    
    #Define the crs for the coral data
    crs_wgs84 = CRS.from_epsg(4326)  #WGS84 (lat/lon coordinates)

    transformer_wgs84_to_sinmod = Transformer.from_crs(crs_wgs84, crs_sinmod, always_xy=True)

    #Transform coral coordinates from WGS84 to SINMOD
    coral_x, coral_y = transformer_wgs84_to_sinmod.transform(coral_data['lengdegrad'], coral_data['breddegrad'])

    #Save coral_x and coral_y as new columns
    coral_data.loc[:, 'x'] = coral_x
    coral_data.loc[:, 'y'] = coral_y

    return coral_data

transformed_coral_data = transform_coral_coordinates(gdf_all_coral_files, crs_sinmod)

output_directory = '/cluster/home/maikents/features_midnor_2019/coral_data/'
output_file = 'midnor_coral_data_sinmod_grid.parquet'

os.makedirs(output_directory, exist_ok=True)


#Save the dataframe to a parquet file
transformed_coral_data.to_parquet(os.path.join(output_directory, output_file), index=False)



In [ ]:
#How to load coral data later:
#coral_filepath = '/output_directory/output_file'
#coral_data = pd.read_parquet(coral_filepath)

In [25]:
coral_file_sinmod_grid = '/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/midnor_total_coral_data_processed.parquet'
coral_ds_sinmod_grid = pd.read_parquet(coral_file_sinmod_grid)

In [2]:
coral_file_emod_grid = '/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/midnor_total_coral_data_filtered_UTM32N.parquet'
coral_ds_emod_grid = pd.read_parquet(coral_file_emod_grid)

In [ ]:
#OPTIONAL

fig, ax = plt.subplots(figsize=(10, 10))

#Plot the reprojected SINMOD data (bottom_temperature, for example)
SINMOD_features_reprojected['bottom_temperature_sundahl_features'].isel(stat=0).plot(ax=ax, cmap='viridis', alpha=0.6)

#Plot coral locations
ax.scatter(coral_ds_emod_grid['x'], coral_ds_emod_grid['y'], color='red', s=10, label='Coral Locations')

ax.set_title('Coral Locations on Reprojected SINMOD Temperature Features')
plt.xlabel("xc (SINMOD grid coordinates)")
plt.ylabel("yc (SINMOD grid coordinates)")
plt.legend()
plt.show()

In [2]:
coral_filepath_emod = '/cluster/home/maikents/features_midnor_2019/coral_data/midnor_coral_data_emod_grid.parquet'
coral_data_emod = pd.read_parquet(coral_filepath_emod)

In [4]:
print(type(coral_data_emod))
print(coral_data_emod)

<class 'pandas.core.frame.DataFrame'>
                 x             y
0    541563.035108  7.183257e+06
1    574769.093767  7.053834e+06
2    448510.482064  7.072666e+06
3    518562.649871  7.129859e+06
4    475671.223693  7.111198e+06
..             ...           ...
836  503204.433394  7.472706e+06
837  530483.004326  7.455677e+06
838  530353.382786  7.455445e+06
839  530295.396581  7.455336e+06
840  510370.164806  7.466556e+06

[841 rows x 2 columns]


In [5]:
coral_data_emod.to_csv('/cluster/home/maikents/sinmod_features_masters/coral_points_for_arcgis.csv', index=False)
